In [1]:
import dronekit_sitl
import time 
from __future__ import print_function
from pymavlink import mavutil 
import math
from dronekit import connect, VehicleMode, LocationGlobalRelative, LocationGlobal, Command


In [2]:
vehicle = connect('127.0.0.1:14550', wait_ready=True)

CRITICAL:autopilot:APM:Copter V3.3 (d6053245)
CRITICAL:autopilot:Frame: QUAD


In [4]:
print(" GPS: %s",vehicle.gps_0)
print("Armed: %s",vehicle.armed)

 GPS: %s GPSInfo:fix=3,num_sat=10
Armed: %s False


In [40]:
# vehicle.mode = VehicleMode("GUIDED")
# time.sleep(10)
# print(vehicle.mode)

VehicleMode:GUIDED


In [39]:
# vehicle.armed =False
# time.sleep(10)
# print("Armed: %s",vehicle.armed)

Armed: %s False


In [41]:
# vehicle.gimbal.rotate(0, 0, 0)
# time.sleep(10)
# print("云台位置 %s",vehicle.gimbal)

云台位置 %s Gimbal: pitch=None, roll=None, yaw=None


In [6]:

arm_and_takeoff(20)
vehicle.airspeed = 3
    
print("无人机前往第一个航点采集土壤")
point1 = LocationGlobalRelative(-35.361354, 149.165218, 20)
vehicle.simple_goto(point1)
time.sleep(30)
print("无人机前往第二个航点采集土壤")
point1 = LocationGlobalRelative(-35.361354, 149.165218, 20)
vehicle.simple_goto(point1)
time.sleep(30)

print("无人机准备返航")
vehicle.mode = VehicleMode("RTL")

vehicle.close()

Arming motors
进入制导模式
10秒后准备起飞


ERROR:autopilot:ARMING MOTORS
CRITICAL:autopilot:GROUND START
CRITICAL:autopilot:Initialising APM...


起飞!
现在高度是:  0.0
现在高度是:  0.0
现在高度是:  0.17
现在高度是:  1.37
现在高度是:  3.2
现在高度是:  5.79
现在高度是:  7.99
现在高度是:  10.19
现在高度是:  12.94
现在高度是:  15.15
现在高度是:  17.32
现在高度是:  18.97
现在高度是:  19.65
到达指定高度
无人机前往第一个航点采集土壤
无人机前往第二个航点采集土壤
无人机准备返航


In [73]:
def send_ned_velocity(velocity_x, velocity_y, velocity_z, duration):
#为无人机下达移动指令
    msg = vehicle.message_factory.set_position_target_local_ned_encode(
        0,       # time_boot_ms (not used)
        0, 0,    # target system, target component
        mavutil.mavlink.MAV_FRAME_LOCAL_NED, # frame
        0b0000111111000111, # type_mask (only speeds enabled)
        0, 0, 0, # x, y, z positions (not used)
        velocity_x, velocity_y, velocity_z, # x, y, z velocity in m/s
        0, 0, 0, # x, y, z acceleration (not supported yet, ignored in GCS_Mavlink)
        0, 0)    # yaw, yaw_rate (not supported yet, ignored in GCS_Mavlink)


    # send command to vehicle on 1 Hz cycle
    for x in range(0,duration):
        vehicle.send_mavlink(msg)
        time.sleep(1)

In [5]:
#起飞
def arm_and_takeoff(aTargetAltitude):
    while not vehicle.is_armable:
        print("无人机没准备好，10秒准备起飞")
        time.sleep(1)

    print("Arming motors")
    print("进入制导模式")
    vehicle.mode    = VehicleMode("GUIDED")
    print("10秒后准备起飞")
    # time.sleep(10)
    vehicle.armed   = True
    time.sleep(5)

    # Confirm vehicle armed before attempting to take off
    while not vehicle.armed:
        print("故障")
        time.sleep(1)

    print("起飞!")
    vehicle.simple_takeoff(aTargetAltitude) 
    while True:
        print("现在高度是: ", vehicle.location.global_relative_frame.alt)
        if vehicle.location.global_relative_frame.alt>=aTargetAltitude*0.95:
            print("到达指定高度")
            break
        time.sleep(1)



#偏航角
def condition_yaw(heading, relative=False):
    if relative:
        is_relative = 1
    else:
        is_relative = 0 
    #传递信息偏航角
    msg = vehicle.message_factory.command_long_encode(
        0, 0,    # target system, target component
        mavutil.mavlink.MAV_CMD_CONDITION_YAW, #command
        0, #confirmation
        heading,    # param 1, yaw in degrees
        0,          # param 2, yaw speed deg/s
        1,          # param 3, direction -1 ccw, 1 cw
        is_relative, # param 4, relative offset 1, absolute angle 0
        0, 0, 0)    # param 5 ~ 7 not used
    # 发送命令
    vehicle.send_mavlink(msg)

#调相机，精度维度高度，相机的对的方向
def set_roi(location):
    msg = vehicle.message_factory.command_long_encode(
        0, 0,    # target system, target component
        mavutil.mavlink.MAV_CMD_DO_SET_ROI, #command
        0, #confirmation
        0, 0, 0, 0, #params 1-4
        location.lat,
        location.lon,
        location.alt
        )
    # send command to vehicle
    vehicle.send_mavlink(msg)
#####################坐标###########################
    
#定点飞行，返回坐标
def get_location_metres(original_location, dNorth, dEast):

    earth_radius = 6378137.0 #地球的半径
    #Coordinate offsets in radians
    dLat = dNorth/earth_radius
    dLon = dEast/(earth_radius*math.cos(math.pi*original_location.lat/180))

    #十进制的新位置
    newlat = original_location.lat + (dLat * 180/math.pi)
    newlon = original_location.lon + (dLon * 180/math.pi)
    if type(original_location) is LocationGlobal:
        targetlocation=LocationGlobal(newlat, newlon,original_location.alt)
    elif type(original_location) is LocationGlobalRelative:
        targetlocation=LocationGlobalRelative(newlat, newlon,original_location.alt)
    else:
        raise Exception("Invalid Location object passed")
        
    return targetlocation;


def get_distance_metres(aLocation1, aLocation2):
#返回两个LocationGlobal对象之间的地面距离，单位为米。
    dlat = aLocation2.lat - aLocation1.lat
    dlong = aLocation2.lon - aLocation1.lon
    return math.sqrt((dlat*dlat) + (dlong*dlong)) * 1.113195e5


def get_bearing(aLocation1, aLocation2):
#返回作为参数传递的两个LocationGlobal对象之间的方位。
    off_x = aLocation2.lon - aLocation1.lon
    off_y = aLocation2.lat - aLocation1.lat
    bearing = 90.00 + math.atan2(-off_y, off_x) * 57.2957795
    if bearing < 0:
        bearing += 360.00
    return bearing;



#######################定点手调#####################

def goto_position_target_global_int(aLocation):
#飞到指定的LocationGlobalX、Y、Z
    msg = vehicle.message_factory.set_position_target_global_int_encode(
        0,       # time_boot_ms (not used)
        0, 0,    # target system, target component
        mavutil.mavlink.MAV_FRAME_GLOBAL_RELATIVE_ALT_INT, # frame
        0b0000111111111000, # type_mask (only speeds enabled)
        aLocation.lat*1e7, # lat_int - X Position in WGS84 frame in 1e7 * meters
        aLocation.lon*1e7, # lon_int - Y Position in WGS84 frame in 1e7 * meters
        aLocation.alt, # alt - Altitude in meters in AMSL altitude, not WGS84 if absolute or relative, above terrain if GLOBAL_TERRAIN_ALT_INT
        0, # X velocity in NED frame in m/s
        0, # Y velocity in NED frame in m/s
        0, # Z velocity in NED frame in m/s
        0, 0, 0, # afx, afy, afz acceleration (not supported yet, ignored in GCS_Mavlink)
        0, 0)    # yaw, yaw_rate (not supported yet, ignored in GCS_Mavlink) 
    # send command to vehicle
    vehicle.send_mavlink(msg)



def goto_position_target_local_ned(north, east, down):
#飞到指定位置，相对原点
    msg = vehicle.message_factory.set_position_target_local_ned_encode(
        0,       # time_boot_ms (not used)
        0, 0,    # target system, target component
        mavutil.mavlink.MAV_FRAME_LOCAL_NED, # frame
        0b0000111111111000, # type_mask (only positions enabled)
        north, east, down, # x, y, z positions (or North, East, Down in the MAV_FRAME_BODY_NED frame
        0, 0, 0, # x, y, z velocity in m/s  (not used)
        0, 0, 0, # x, y, z acceleration (not supported yet, ignored in GCS_Mavlink)
        0, 0)    # yaw, yaw_rate (not supported yet, ignored in GCS_Mavlink) 
    # send command to vehicle
    vehicle.send_mavlink(msg)



def goto(dNorth, dEast, gotoFunction=vehicle.simple_goto):

    currentLocation = vehicle.location.global_relative_frame
    targetLocation = get_location_metres(currentLocation, dNorth, dEast)
    targetDistance = get_distance_metres(currentLocation, targetLocation)
    gotoFunction(targetLocation)
    
    #print "DEBUG: targetLocation: %s" % targetLocation
    #print "DEBUG: targetLocation: %s" % targetDistance

    while vehicle.mode.name=="GUIDED": #Stop action if we are no longer in guided mode.
        #print "DEBUG: mode: %s" % vehicle.mode.name
        remainingDistance=get_distance_metres(vehicle.location.global_relative_frame, targetLocation)
        print("Distance to target: ", remainingDistance)
        if remainingDistance<=targetDistance*0.01: #Just below target, in case of undershoot.
            print("Reached target")
            break;
        time.sleep(2)



########################速度分量####################

def send_ned_velocity(velocity_x, velocity_y, velocity_z, duration):
#调速度
    msg = vehicle.message_factory.set_position_target_local_ned_encode(
        0,       # time_boot_ms (not used)
        0, 0,    # target system, target component
        mavutil.mavlink.MAV_FRAME_LOCAL_NED, # frame
        0b0000111111000111, # type_mask (only speeds enabled)
        0, 0, 0, # x, y, z positions (not used)
        velocity_x, velocity_y, velocity_z, # x, y, z velocity in m/s
        0, 0, 0, # x, y, z acceleration (not supported yet, ignored in GCS_Mavlink)
        0, 0)    # yaw, yaw_rate (not supported yet, ignored in GCS_Mavlink) 

    # send command to vehicle on 1 Hz cycle
    for x in range(0,duration):
        vehicle.send_mavlink(msg)
        time.sleep(1)
    
    


def send_global_velocity(velocity_x, velocity_y, velocity_z, duration):
#指定方向
    msg = vehicle.message_factory.set_position_target_global_int_encode(
        0,       # time_boot_ms (not used)
        0, 0,    # target system, target component
        mavutil.mavlink.MAV_FRAME_GLOBAL_RELATIVE_ALT_INT, # frame
        0b0000111111000111, # type_mask (only speeds enabled)
        0, # lat_int - X Position in WGS84 frame in 1e7 * meters
        0, # lon_int - Y Position in WGS84 frame in 1e7 * meters
        0, # alt - Altitude in meters in AMSL altitude(not WGS84 if absolute or relative)
        # altitude above terrain if GLOBAL_TERRAIN_ALT_INT
        velocity_x, # X velocity in NED frame in m/s
        velocity_y, # Y velocity in NED frame in m/s
        velocity_z, # Z velocity in NED frame in m/s
        0, 0, 0, # afx, afy, afz acceleration (not supported yet, ignored in GCS_Mavlink)
        0, 0)    # yaw, yaw_rate (not supported yet, ignored in GCS_Mavlink) 

    # send command to vehicle on 1 Hz cycle
    for x in range(0,duration):
        vehicle.send_mavlink(msg)
        time.sleep(1)    


In [74]:
arm_and_takeoff(20)
SOUTH=-2
UP=-0.5 
DURATION=100
send_ned_velocity(SOUTH,0,UP,DURATION)

print("无人机准备返航")
vehicle.mode = VehicleMode("RTL")

vehicle.close()

Arming motors
进入制导模式
10秒后准备起飞
起飞!
现在高度是:  19.98
到达指定高度
无人机准备返航


In [76]:
"""
使用标准Vehicle.simple_goto()方法飞行三角形路径。该方法通过自定义的“goto”间接调用，该“goto”允许目标位置为
以米为单位指明距离当前位置(北/东)
"""
print("Vehicle.simple_goto()函数测试")
print("Set groundspeed to 5m/s.")
vehicle.groundspeed=5

print("北纬 80 西经 50")
goto(-35.361354, 149.165218)
print("无人机准备返航")
vehicle.mode = VehicleMode("RTL")

vehicle.close()



Arming motors
进入制导模式
10秒后准备起飞
起飞!
现在高度是:  19.98
到达指定高度
Vehicle.simple_goto()函数测试
Set groundspeed to 5m/s.
北纬 80 西经 50
无人机准备返航


In [ ]:
"""
使用SET_POSITION_TARGET_GLOBAL_INT命令执行路径，并指定
目标位置。
"""
arm_and_takeoff(5)


print("Set groundspeed to 5m/s.")
vehicle.groundspeed = 5
goto(-100, -130, goto_position_target_global_int)

print("Set groundspeed to 15m/s (max).")
vehicle.groundspeed = 15
print("Position South 0 East 200")
goto(0, 260, goto_position_target_global_int)

print("Set airspeed to 10m/s (max).")
vehicle.airspeed = 10

print("Position North 100 West 130")
goto(100, -130, goto_position_target_global_int)

In [ ]:
"""
使用SET_POSITION_TARGET_LOCAL_NED命令在50m的方形路径上飞行
并指定目标位置。该命令从goto_position_target_local_ned()调用。
代码还通过' set_roi() '方法设置感兴趣的区域(MAV_CMD_DO_SET_ROI)。这就指向
相机云台在选定的位置(在这种情况下，它对准整个无人机指向ROI)。
"""
print("SQUARE path using SET_POSITION_TARGET_LOCAL_NED and position parameters")
DURATION = 20 #Set duration for each segment.

print("North 50m, East 0m, 10m altitude for %s seconds" % DURATION)
goto_position_target_local_ned(50,0,-10)
print("Point ROI at current location (home position)") 

set_roi(vehicle.location.global_relative_frame)
time.sleep(DURATION)

print("North 50m, East 50m, 10m altitude")
goto_position_target_local_ned(50,50,-10)
time.sleep(DURATION)

print("Point ROI at current location")
set_roi(vehicle.location.global_relative_frame)

print("North 0m, East 50m, 10m altitude")
goto_position_target_local_ned(0,50,-10)
time.sleep(DURATION)

print("North 0m, East 0m, 10m altitude")
goto_position_target_local_ned(0,0,-10)
time.sleep(DURATION)

In [ ]:
"""
使用速度向量在SQUARE路径控制无人机飞行
线程休眠一段时间(DURATION)，这段时间定义了将要运行的距离。
代码还在每个段中使用' set_yaw() '方法设置偏航(MAV_CMD_CONDITION_YAW)
这样前部就会指向行驶的方向
"""
NORTH = 2
SOUTH = -2

# Note for vy:
# vy > 0 => fly East
# vy < 0 => fly West
EAST = 2
WEST = -2

# Note for vz: 
# vz < 0 => ascend
# vz > 0 => descend
UP = -0.5
DOWN = 0.5


# Square path using velocity
print("SQUARE path using SET_POSITION_TARGET_LOCAL_NED and velocity parameters")

print("Yaw 180 absolute (South)")
condition_yaw(180)

print("Velocity South & up")
send_ned_velocity(SOUTH,0,UP,DURATION)
send_ned_velocity(0,0,0,1)


print("Yaw 270 absolute (West)")
condition_yaw(270)

print("Velocity West & down")
send_ned_velocity(0,WEST,DOWN,DURATION)
send_ned_velocity(0,0,0,1)


print("Yaw 0 absolute (North)")
condition_yaw(0)

print("Velocity North")
send_ned_velocity(NORTH,0,0,DURATION)
send_ned_velocity(0,0,0,1)


print("Yaw 90 absolute (East)")
condition_yaw(90)

print("Velocity East")
send_ned_velocity(0,EAST,0,DURATION)
send_ned_velocity(0,0,0,1)

In [78]:
"""
使用速度向量在DIAMOND路径上飞行、

线程休眠一段时间(DURATION)，这段时间定义了将要运行的距离。

代码使用使用相对标题的' set_yaw() '方法设置偏航(MAV_CMD_CONDITION_YAW)
这样车辆的前部就会指向行驶的方向。

在第二段的末尾，代码将设置当前点的新home位置。
"""
arm_and_takeoff(5)
DURATION=4
SOUTH=1
NORTH=1
UP=1
DOWN=1
EAST=1
WEST=1

print("DIAMOND path using SET_POSITION_TARGET_GLOBAL_INT and velocity parameters")
# vx, vy are parallel to North and East (independent of the vehicle orientation)

print("Yaw 225 absolute")
condition_yaw(225)

print("Velocity South, West and Up")
send_global_velocity(SOUTH,WEST,UP,DURATION)
send_global_velocity(0,0,0,1)


print("Yaw 90 relative (to previous yaw heading)")
condition_yaw(90,relative=True)

print("Velocity North, West and Down")
send_global_velocity(NORTH,WEST,DOWN,DURATION)
send_global_velocity(0,0,0,1)

print("Set new home location to current location")
vehicle.home_location=vehicle.location.global_frame
print("Get new home location")
#This reloads the home location in DroneKit and GCSs
cmds = vehicle.commands
cmds.download()
cmds.wait_ready()
print(" Home Location: %s" % vehicle.home_location)


print("Yaw 90 relative (to previous yaw heading)")
condition_yaw(90,relative=True)

print("Velocity North and East")
send_global_velocity(NORTH,EAST,0,DURATION)
send_global_velocity(0,0,0,1)


print("Yaw 90 relative (to previous yaw heading)")
condition_yaw(90,relative=True)

print("Velocity South and East")
send_global_velocity(SOUTH,EAST,0,DURATION)
send_global_velocity(0,0,0,1)

print("无人机准备返航")
vehicle.mode = VehicleMode("RTL")

vehicle.close()

In [ ]:
# vehicle.close()
# sitl.stop()
# print("Completed")

In [79]:
# #Point the gimbal straight down
# vehicle.gimbal.rotate(-90, 0, 0)
# time.sleep(10)

# #Set the camera to track the current home position.
# vehicle.gimbal.target_location(vehicle.home_location)
# time.sleep(10)


In [28]:
# #Callback to print the location in global frames. 'value' is the updated value
# def location_callback(self, attr_name, value):
#     print("Location (Global): ", value)
#      # Add a callback `location_callback` for the `global_frame` attribute.
#     vehicle.add_attribute_listener('location.global_frame', location_callback)

#  # Wait 2s so callback can be notified before the observer is removed
#     time.sleep(2)

#  # Remove observer - specifying the attribute and previously registered callback function
#     vehicle.remove_message_listener('location.global_frame', location_callback)




In [ ]:
#返回LocationGlobal对象，该对象包含从节点开始的纬度/经度“dNorth”和“dEast”米
#指定“original_location”

def get_location_metres(original_location, dNorth, dEast):

    earth_radius=6378137.0 
    dLat = dNorth/earth_radius
    dLon = dEast/(earth_radius*math.cos(math.pi*original_location.lat/180))

    newlat = original_location.lat + (dLat * 180/math.pi)
    newlon = original_location.lon + (dLon * 180/math.pi)
    return LocationGlobal(newlat, newlon,original_location.alt)

#返回两个LocationGlobal对象之间的地面距离，单位为米。
def get_distance_metres(aLocation1, aLocation2):
    
    dlat = aLocation2.lat - aLocation1.lat
    dlong = aLocation2.lon - aLocation1.lon
    return math.sqrt((dlat*dlat) + (dlong*dlong)) * 1.113195e5

In [3]:
#以米为单位获取到当前航路点的距离。
def distance_to_current_waypoint():

    nextwaypoint = vehicle.commands.next
    if nextwaypoint==0:
        return None
    missionitem=vehicle.commands[nextwaypoint-1] #commands are zero indexed
    lat = missionitem.x
    lon = missionitem.y
    alt = missionitem.z
    targetWaypointLocation = LocationGlobalRelative(lat,lon,alt)
    distancetopoint = get_distance_metres(vehicle.location.global_frame, targetWaypointLocation)
    return distancetopoint

#从无人机下载任务
def download_mission():

    cmds = vehicle.commands
    cmds.download()
    cmds.wait_ready() # wait until download is complete.


#在当前任务中增加一个起飞命令和四个航路点命令。
#这些路径点被定位为围绕指定的LocationGlobal (aLocation)形成边长为2*aSize的正方形。
def adds_square_mission(aLocation, aSize):

    cmds = vehicle.commands

    print(" Clear any existing commands")
    cmds.clear() 
    
    print(" Define/add new commands.")
    
    #Add MAV_CMD_NAV_TAKEOFF command. This is ignored if the vehicle is already in the air.
    cmds.add(Command( 0, 0, 0, mavutil.mavlink.MAV_FRAME_GLOBAL_RELATIVE_ALT, mavutil.mavlink.MAV_CMD_NAV_TAKEOFF, 0, 0, 0, 0, 0, 0, 0, 0, 10))

    #Define the four MAV_CMD_NAV_WAYPOINT locations and add the commands
    point1 = get_location_metres(aLocation, aSize, -aSize)
    point2 = get_location_metres(aLocation, aSize, aSize)
    point3 = get_location_metres(aLocation, -aSize, aSize)
    point4 = get_location_metres(aLocation, -aSize, -aSize)
    cmds.add(Command( 0, 0, 0, mavutil.mavlink.MAV_FRAME_GLOBAL_RELATIVE_ALT, mavutil.mavlink.MAV_CMD_NAV_WAYPOINT, 0, 0, 0, 0, 0, 0, point1.lat, point1.lon, 11))
    cmds.add(Command( 0, 0, 0, mavutil.mavlink.MAV_FRAME_GLOBAL_RELATIVE_ALT, mavutil.mavlink.MAV_CMD_NAV_WAYPOINT, 0, 0, 0, 0, 0, 0, point2.lat, point2.lon, 12))
    cmds.add(Command( 0, 0, 0, mavutil.mavlink.MAV_FRAME_GLOBAL_RELATIVE_ALT, mavutil.mavlink.MAV_CMD_NAV_WAYPOINT, 0, 0, 0, 0, 0, 0, point3.lat, point3.lon, 13))
    cmds.add(Command( 0, 0, 0, mavutil.mavlink.MAV_FRAME_GLOBAL_RELATIVE_ALT, mavutil.mavlink.MAV_CMD_NAV_WAYPOINT, 0, 0, 0, 0, 0, 0, point4.lat, point4.lon, 14))
    #add dummy waypoint "5" at point 4 (lets us know when have reached destination)
    cmds.add(Command( 0, 0, 0, mavutil.mavlink.MAV_FRAME_GLOBAL_RELATIVE_ALT, mavutil.mavlink.MAV_CMD_NAV_WAYPOINT, 0, 0, 0, 0, 0, 0, point4.lat, point4.lon, 14))    

    print(" Upload new commands to vehicle")
    cmds.upload()


In [8]:
print('Create a new mission (for current location)')
adds_square_mission(vehicle.location.global_frame,50)


#起飞
arm_and_takeoff(10)

print("开始执行采土任务")
# 设置第一任务点
vehicle.commands.next=0

# 自动模式
vehicle.mode = VehicleMode("AUTO")


#使用distance_to_current_waypoint()，这是一个飞正方形的函数

while True:
    nextwaypoint=vehicle.commands.next
    print('Distance to waypoint (%s): %s' % (nextwaypoint, distance_to_current_waypoint()))
  
    if nextwaypoint==3: #Skip to next waypoint
        print('Skipping to Waypoint 5 when reach waypoint 3')
        vehicle.commands.next = 5
    if nextwaypoint==5: #Dummy waypoint - as soon as we reach waypoint 4 this is true and we exit.
        print("Exit 'standard' mission when start heading to final waypoint (5)")
        break;
    time.sleep(1)
    

print('开始返航')
vehicle.mode = VehicleMode("RTL")


print("关闭引擎")
vehicle.close()



In [9]:
def readmission(aFileName):
#读取任务
    print("\nReading mission from file: %s" % aFileName)
    cmds = vehicle.commands
    missionlist=[]
    with open(aFileName) as f:
        for i, line in enumerate(f):
            if i==0:
                if not line.startswith('QGC WPL 110'):
                    raise Exception('File is not supported WP version')
            else:
                linearray=line.split('\t')
                ln_index=int(linearray[0])
                ln_currentwp=int(linearray[1])
                ln_frame=int(linearray[2])
                ln_command=int(linearray[3])
                ln_param1=float(linearray[4])
                ln_param2=float(linearray[5])
                ln_param3=float(linearray[6])
                ln_param4=float(linearray[7])
                ln_param5=float(linearray[8])
                ln_param6=float(linearray[9])
                ln_param7=float(linearray[10])
                ln_autocontinue=int(linearray[11].strip())
                cmd = Command( 0, 0, 0, ln_frame, ln_command, ln_currentwp, ln_autocontinue, ln_param1, ln_param2, ln_param3, ln_param4, ln_param5, ln_param6, ln_param7)
                missionlist.append(cmd)
    return missionlist


def upload_mission(aFileName):
#上传任务
    #Read mission from file
    missionlist = readmission(aFileName)
    
    print("\nUpload mission from a file: %s" % aFileName)
    #Clear existing mission from vehicle
    print(' Clear mission')
    cmds = vehicle.commands
    cmds.clear()
    #Add new mission to vehicle
    for command in missionlist:
        cmds.add(command)
    print(' Upload mission')
    vehicle.commands.upload()


def download_mission():
#下载任务并做成一个list
    print(" Download mission from vehicle")
    missionlist=[]
    cmds = vehicle.commands
    cmds.download()
    cmds.wait_ready()
    for cmd in cmds:
        missionlist.append(cmd)
    return missionlist

def save_mission(aFileName):
#保存航点到txt文件
    print("\nSave mission from Vehicle to file: %s" % aFileName)    
    #Download mission from vehicle
    missionlist = download_mission()
    #Add file-format information
    output='QGC WPL 110\n'
    #Add home location as 0th waypoint
    home = vehicle.home_location
    output+="%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n" % (0,1,0,16,0,0,0,0,home.lat,home.lon,home.alt,1)
    #Add commands
    for cmd in missionlist:
        commandline="%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n" % (cmd.seq,cmd.current,cmd.frame,cmd.command,cmd.param1,cmd.param2,cmd.param3,cmd.param4,cmd.x,cmd.y,cmd.z,cmd.autocontinue)
        output+=commandline
    with open(aFileName, 'w') as file_:
        print(" Write mission to file")
        file_.write(output)
        
        
def printfile(aFileName):
#打印文件任务
    print("\nMission file: %s" % aFileName)
    with open(aFileName) as f:
        for line in f:
            print(' %s' % line.strip())        

In [11]:
import_mission_filename = 'mpmission.txt'
export_mission_filename = 'exportedmission.txt'


#Upload mission from file
upload_mission(import_mission_filename)

#Download mission we just uploaded and save to a file
save_mission(export_mission_filename)

#Close vehicle object before exiting script
print("Close vehicle object")
vehicle.close()


# if sitl is not None:
#     sitl.stop()


print("\nShow original and uploaded/downloaded files:")
#Print original file (for demo purposes only)
printfile(import_mission_filename)
#Print exported file (for demo purposes only)
printfile(export_mission_filename)


Reading mission from file: mpmission.txt

Upload mission from a file: mpmission.txt
 Clear mission
 Upload mission


CRITICAL:autopilot:flight plan received



Save mission from Vehicle to file: exportedmission.txt
 Download mission from vehicle
 Write mission to file
Close vehicle object

Show original and uploaded/downloaded files:

Mission file: mpmission.txt
 QGC WPL 110
 0	1	0	16	0	0	0	0	-35.363262	149.165237	584.000000	1
 1	0	0	22	0.000000	0.000000	0.000000	0.000000	-35.361988	149.163753	100.000000	1
 2	0	0	16	0.000000	0.000000	0.000000	0.000000	-35.361992	149.163593	100.000000	1
 3	0	0	16	0.000000	0.000000	0.000000	0.000000	-35.363812	149.163609	100.000000	1
 4	0	0	16	0.000000	0.000000	0.000000	0.000000	-35.363768	149.166055	100.000000	1
 5	0	0	16	0.000000	0.000000	0.000000	0.000000	-35.361835	149.166012	100.000000	1
 6	0	0	16	0.000000	0.000000	0.000000	0.000000	-35.362150	149.165046	100.000000	1

Mission file: exportedmission.txt
 QGC WPL 110
 0	1	0	16	0	0	0	0	31.93865966796875	118.78984832763672	2.990000009536743	1
 1	0	0	16	0.0	0.0	0.0	0.0	-35.36326217651367	149.1652374267578	584.0	1
 2	0	0	22	0.0	0.0	0.0	0.0	-35.36198806762695	149

In [ ]:
#PWM 
# import RPi.GPIO as GPIO                 #引入RPi.GPIO库函数命名为GPIO
# import tty,sys,select,termios
 
# def getKey(settings):
#     tty.setraw(sys.stdin.fileno())
#     rlist = select.select([sys.stdin],[],[],0.1)
    
#     if rlist:
#         key = sys.stdin.read(1)
#     else:
#         key = ""
    
#     termios.tcsetattr(sys.stdin,termios.TCSADRAIN,settings)
#     return key
 
# # BOARD编号方式，基于插座引脚编号
# GPIO.setmode(GPIO.BOARD)                #将GPIO编程方式设置为BOARD模式
 
# #接口定义，注意对照树莓派的引脚图
# INT1 = 11                              
# INT2 = 12
# INT3 = 13                              
# INT4 = 15
 
# #输出模式
# GPIO.setup(INT1,GPIO.OUT)
# GPIO.setup(INT2,GPIO.OUT)
# GPIO.setup(INT3,GPIO.OUT)
# GPIO.setup(INT4,GPIO.OUT)
 
 
# #前进
# def Forward():
#     GPIO.output(INT1,GPIO.HIGH)
#     GPIO.output(INT2,GPIO.LOW)
#     GPIO.output(INT3,GPIO.HIGH)
#     GPIO.output(INT4,GPIO.LOW)
# #后退
# def Back():
#     GPIO.output(INT1,GPIO.LOW)
#     GPIO.output(INT2,GPIO.HIGH)
#     GPIO.output(INT3,GPIO.LOW)
#     GPIO.output(INT4,GPIO.HIGH)
# #右转
# def Right():
#     GPIO.output(INT1,GPIO.HIGH)
#     GPIO.output(INT2,GPIO.LOW)
#     GPIO.output(INT3,GPIO.LOW)
#     GPIO.output(INT4,GPIO.LOW)
# #左转    
# def Left():
#     GPIO.output(INT1,GPIO.LOW)
#     GPIO.output(INT2,GPIO.LOW)
#     GPIO.output(INT3,GPIO.HIGH)
#     GPIO.output(INT4,GPIO.LOW)
# #停止    
# def Stop():
#     GPIO.output(INT1,GPIO.LOW)
#     GPIO.output(INT2,GPIO.LOW)
#     GPIO.output(INT3,GPIO.LOW)
#     GPIO.output(INT4,GPIO.LOW)
 
# while(1):
#     setting = termios.tcgetattr(sys.stdin)
#     InPut = getKey(setting)
#     if InPut == "w":
#         Forward()
#     elif InPut == "s":
#         Stop()
#     elif InPut == "x":
#         Back()
#     elif InPut == "a":
#         Left()
#     elif InPut == "d":
#         Right()
#     else:
#         print("Input Error,Please give a true index!!")
